In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)


def get_ret_year(mf_rets, st, ed):
    mf_rets_year = mf_rets[(mf_rets['date']<=ed) & (mf_rets['date']>=st)]
    date_ls = sorted(list(set(mf_rets_year['date'])))
    mf_rets_dicts = mf_rets_year.to_dict('records')
    rets_year_dict = {}
    for dict_i in mf_rets_dicts:
        crsp_fundno = dict_i['crsp_fundno']
        date = dict_i['date']
        ret = dict_i['mret']
        if crsp_fundno not in rets_year_dict:
            rets_year_dict[crsp_fundno] = {}
        rets_year_dict[crsp_fundno][date] = ret
    
    fundno_rets = {}
    NAN = float('nan')
    for fund_i in rets_year_dict:
        dict_i = rets_year_dict[fund_i]
        rest_ls = [NAN for i in range(len(date_ls))]
        for dete_i in dict_i:
            ind = date_ls.index(dete_i)
            rest_ls[ind] = dict_i[dete_i]
        fundno_rets[fund_i] = rest_ls
    rets_year = pd.DataFrame.from_dict(fundno_rets)
    #rets_year = rets_year.dropna(axis=1, how='any')
    return rets_year


def get_ret_year1(mf_rets, st, ed, st1, ed1):
    mf_rets_year = mf_rets[(mf_rets['date']<=ed1) & (mf_rets['date']>=st)]
    date_ls = sorted(list(set(mf_rets_year['date'])))
    mf_rets_dicts = mf_rets_year.to_dict('records')
    rets_year_dict = {}
    for dict_i in mf_rets_dicts:
        crsp_fundno = dict_i['crsp_fundno']
        date = dict_i['date']
        ret = dict_i['mret']
        if crsp_fundno not in rets_year_dict:
            rets_year_dict[crsp_fundno] = {}
        rets_year_dict[crsp_fundno][date] = ret
    
    fundno_rets = {}
    NAN = float('nan')
    for fund_i in rets_year_dict:
        dict_i = rets_year_dict[fund_i]
        rest_ls = [NAN for i in range(len(date_ls))]
        for dete_i in dict_i:
            ind = date_ls.index(dete_i)
            rest_ls[ind] = dict_i[dete_i]
        fundno_rets[fund_i] = rest_ls
    rets_year = pd.DataFrame.from_dict(fundno_rets)
    rets_year = rets_year.dropna(axis=1, how='any')
    rets_year['date'] = date_ls
    rets_year1 = rets_year[(rets_year['date']<=ed) & (rets_year['date']>=st)]
    rets_year2 = rets_year[(rets_year['date']<=ed1) & (rets_year['date']>=st1)]
    rets_year1 = rets_year1.drop('date', axis=1)
    rets_year2 = rets_year2.drop('date', axis=1)
    return rets_year1, rets_year2


def get_qu_ls(quts, fund_retavg):
    ls = [[], [], [], [], [], [], [], [], [], []]
    for id_i in fund_retavg:
        avg = fund_retavg[id_i]
        for i in range(10):
            if (avg>=quts[i]) & (avg<quts[i+1]):
                ls[i].append(int(id_i))
    return ls
        
def get_group_ls(fundno_rets_year1):
    fund_retavg = fundno_rets_year1.mean().to_dict()
    ret_avgls = [fund_retavg[i] for i in fund_retavg]
    # from low to high:
    quts = list(np.percentile(ret_avgls, [(i+1)*10 for i in range(9)]))
    quts.insert(0, -8888)
    quts.append(8888)
    portfolios_fundids = get_qu_ls(quts, fund_retavg)
    return portfolios_fundids

def get_returns(ret_i):
    ret_array = ret_i.to_numpy()
    rest1 = ret_array[0, :]
    inval_indx0 = np.isnan(rest1)
    val_indx0 = 1- np.isnan(rest1)
    num_vals = np.sum(val_indx0)  
    ws = np.array([1/num_vals for i in range(rest1.shape[0])])
    ws[inval_indx0] = 0
    rets = []
    for i in range(12):
        rest1 = ret_array[i, :]
        rest11 = ret_array[i, :]
        rest11[np.isnan(rest11)] = 0
        ret_f1 = np.matrix(rest11)*np.matrix(ws).T
        ret_f1 = ret_f1[0, 0]
        rets.append(ret_f1)
        if i <= 10:
            rest2 = ret_array[i+1, :]
            indx_inval  = np.isnan(rest2)
            indx_val = 1 - indx_inval
            asets_vals = (np.array(rest11)/100+1)*np.array(ws)
            asets_vals = asets_vals + np.sum(asets_vals[indx_val])/np.sum(indx_val)
            asets_vals[indx_inval] = 0
            ws = asets_vals/np.sum(asets_vals)
            ws = ws.tolist()
    return rets


def get_pf_rets(pf_ids, fundno_rets_year2):
    ret_ls = []
    for pf_i in pf_ids:
        ret_i = fundno_rets_year2[pf_i]
        rets_pf_i = get_returns(ret_i)
        ret_ls.append(rets_pf_i)
    rets = np.array(ret_ls).T
    return rets


def get_all_rets(mf_rets):
    months  = []
    cols = ['p{}'.format(i+1) for i in range(10)]
    NANs = [float('nan') for i in range(12)]
    for i in tqdm(range(55)):
        st = int(str(1964+i)+'01')
        ed = int(str(1964+i)+'12')
        fundno_rets_year1 = get_ret_year(mf_rets, st, ed)
        fundno_rets_year1 = fundno_rets_year1.dropna(axis=1, how='any')
        pf_group_ls = get_group_ls(fundno_rets_year1)
        st1 = int(str(1964+i+1)+'01')
        ed1 = int(str(1964+i+1)+'12')
        fundno_rets_year2 = get_ret_year(mf_rets, st1, ed1)
        new_cols = set(fundno_rets_year1.columns) - set(fundno_rets_year2.columns)
        for col_i in new_cols:
            fundno_rets_year2[col_i] = NANs
        rets_i = get_pf_rets(pf_group_ls, fundno_rets_year2)
        if i==0:
            rets = rets_i
        else:
            rets = np.vstack((rets, rets_i))
        mon_i = [int(str(1964+i+1)+str(j+1).zfill(2)) for j in range(12)]
        months += mon_i
    df_pf_rets = pd.DataFrame(rets, columns=cols)
    df_pf_rets['date'] = months
    return df_pf_rets


def get_pf_rets1(pf_ids, fundno_rets_year2):
    ret_ls = []
    for pf_i in pf_ids:
        ret_i = fundno_rets_year2[pf_i]
        rets_pf_i = get_returns1(ret_i)
        ret_ls.append(rets_pf_i)
    rets = np.array(ret_ls).T
    return rets


def get_returns1(ret_i):
    ret_array = ret_i.to_numpy()
    rest1 = ret_array[0, :]
    num_vals = rest1.shape[0]
    ws = np.array([1/num_vals for i in range(num_vals)])
    rets = []
    for i in range(12):
        rest1 = ret_array[i, :]
        ret_f1 = np.matrix(rest1)*np.matrix(ws).T
        ret_f1 = ret_f1[0, 0]
        rets.append(ret_f1)
        if i <= 10:
            rest2 = ret_array[i+1, :]
            asets_vals = (np.array(rest1)/100+1)*np.array(ws)
            #print(np.sum(asets_vals))
            ws = asets_vals/np.sum(asets_vals)
            ws = ws.tolist()
    return rets

def get_all_rets1(mf_rets):
    months  = []
    cols = ['p{}'.format(i+1) for i in range(10)]
    NANs = [float('nan') for i in range(12)]
    for i in tqdm(range(54)):
        st = int(str(1964+i)+'01')
        ed = int(str(1964+i)+'12')
        st1 = int(str(1964+i+1)+'01')
        ed1 = int(str(1964+i+1)+'12')
        rets_year1, ret_year2 = get_ret_year1(mf_rets, st, ed, st1, ed1)
        pf_group_ls = get_group_ls(rets_year1)
        rets_i = get_pf_rets1(pf_group_ls, ret_year2)
        if i==0:
            rets = rets_i
        else:
            rets = np.vstack((rets, rets_i))
        mon_i = [int(str(1964+i+1)+str(j+1).zfill(2)) for j in range(12)]
        months += mon_i
    df_pf_rets = pd.DataFrame(rets, columns=cols)
    df_pf_rets['date'] = months
    return df_pf_rets

# 1 process data

In [88]:
mf_rets = pd.read_csv('./original_data/MFrets_1962to2012.csv')
mf_rets = mf_rets.dropna(axis=0, how='any')
years = list(mf_rets['year'])
months = list(mf_rets['month'].astype(str))
dates = []
for i in range(len(years)):
    date_i = str(years[i]) + str(months[i]).zfill(2)
    dates.append(date_i)
mf_rets['date'] = dates
mf_rets['mret'] = mf_rets['mret']*100
mf_rets = mf_rets.drop(['year', 'month'], axis=1)
mf_rets.to_csv('./MF_rets.csv')

In [61]:
mom = pd.read_csv('./original_data/Mom.csv')
df3 = pd.read_csv('./original_data/FF3.csv')
df3 = df3[df3['Unnamed: 0']>=192701]
df3 = df3.reset_index()
df3['Mom'] = mom['Mom']
dates1 = sorted(list(set(dates)))
df3['date'] = df3['Unnamed: 0']
df3 = df3.drop(['Unnamed: 0', 'index'], axis=1)
df3.to_csv('FF4.csv')

# 2 regression:

$$
\begin{gathered}
r_{i t}=\alpha_{i T}+\beta_{i T} \mathrm{VWRF}_{t}+e_{i t} \quad t=1,2, \cdots, T \\
r_{i t}=\alpha_{i T}+b_{i T} \mathrm{RMRF}_{t}+s_{i T} \mathrm{SMB}_{t}+h_{i T} \mathrm{HML}_{t}+e_{i t} \quad t=1,2, \cdots, T \\
r_{i t}=\alpha_{i T}+b_{i T} \mathrm{RMRF}_{t}+s_{i T} \mathrm{SMB}_{t}+h_{i T} \mathrm{HML}_{t}+p_{i T} \mathrm{PR} 1 \mathrm{YR}_{t}+e_{i t} \\
t=1,2, \cdots, T
\end{gathered}
$$

## 2.1 variable meaning:

- $r_{i t}$ is the return on a portfolio in excess of the one-month T-bill return;
- VWRF is the excess return on the CRSP value-weighted portfolio of all NYSE;
- RMRF is the excess return on a value-weighted aggregate market proxy;
- SMB, HML, PR1YR are returns on valueweighted, zero-investment, factor-mimicking portfolios for size, book-to-market equity, and one-year momentum in stock returns.


## 2.2 portfolio  construction:

On January 1 of each year, I form ten equal-weighted portfolios of mutual funds, using reported returns.  I hold the portfolios for one year, then reform them. This yields a time series of monthly returns on each decile portfolio from 1963 to 1993. Funds that disappear during the course of the year are included in the equal-weighted average until they disappear, then the portfolio weights are readjusted appropriately.

Mutual funds are sorted on January 1 each year from 1963 to 1993 into decile portfolios based on their previous calendar year's return. The portfolios are equally weighted monthly so the weights are readjusted whenever a fund disappears. Funds with the highest past one-year return comprise decile 1 and funds with the lowest comprise decile 10. Deciles 1 and 10 are further subdivided into thirds on the same measure. VWRF is the excess return on the CRSP value-weight market proxy.

RMRF, SMB, and HML are Fama and French's (1993) market proxy and factor-mimicking portfolios for size and book-to-market equity. PR1YR is a factor-mimicking portfolio for one-year return momentum. Alpha is the intercept of the Model. The t-statistics are in parentheses.

In [2]:
ff4_df = pd.read_csv('./FF4.csv')
ff4_df = ff4_df.drop('Unnamed: 0', axis=1)
mf_rets = pd.read_csv('./MF_rets.csv')
mf_rets = mf_rets.drop('Unnamed: 0', axis=1)

In [3]:
df_pf_rets = get_all_rets(mf_rets)

100%|███████████████████████████████████████████| 55/55 [00:26<00:00,  2.05it/s]


In [4]:
df_pf_rets = df_pf_rets[df_pf_rets['date']<201901]
df_pf_rets['ls'] = df_pf_rets['p10'] - df_pf_rets['p1']
ff4_df1 = ff4_df[(ff4_df['date']>=196501) & (ff4_df['date']<=201812)]
ff4_df1 = ff4_df1.drop('date',axis=1)
ff4_df1 = ff4_df1.reset_index()

In [5]:
df_pf_rets[ff4_df1.columns[1:]] = ff4_df1[ff4_df1.columns[1:]]
df_pf_rets1 = df_pf_rets.copy()
cls = ['p10', 'p9', 'p8', 'p7', 'p6', 'p5', 'p4', 'p3', 'p2', 'p1']
for cls_i in cls:
    df_pf_rets1[cls_i] = df_pf_rets1[cls_i] - df_pf_rets1['RF']
cls.append('ls')
df_pf_rets1['MK'] = df_pf_rets1['Mkt-RF']
df_pf_rets2 = df_pf_rets1[df_pf_rets1['date']<=199312]
df_pf_rets3 = df_pf_rets1[(df_pf_rets1['date']>199312) & (df_pf_rets1['date']<=201812)]
df_pf_rets1.to_csv('pf_rets1.csv')
df_pf_rets2.to_csv('pf_rets2.csv')
df_pf_rets3.to_csv('pf_rets3.csv')
df_pf_rets1.head(10)

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,date,ls,Mkt-RF,SMB,HML,RF,Mom,MK
0,5.447729,4.463886,4.158650,5.497686,5.861933,4.229543,3.667814,3.084767,4.634757,4.874929,196501,-0.572800,3.54,2.70,0.12,0.28,-1.44,3.54
1,1.373950,1.388286,0.080959,2.740870,2.121548,2.271472,1.643450,3.110765,2.069201,1.905606,196502,0.531656,0.44,3.53,0.11,0.30,0.32,0.44
2,-2.346661,-1.922857,-2.430716,-1.104446,-0.964850,-1.793701,-1.056489,-1.023599,-0.988692,0.067843,196503,2.414504,-1.34,1.88,1.03,0.36,0.15,-1.34
3,3.086380,3.584403,3.060378,4.174120,3.077419,3.022716,3.175290,2.372352,3.688015,3.455459,196504,0.369078,3.11,1.15,0.66,0.31,2.63,3.11
4,-2.125712,-3.212074,-3.409076,-1.007003,-0.730214,-0.762198,-0.634075,-1.212219,-1.261887,-0.946323,196505,1.179389,-0.77,0.13,-1.61,0.31,0.60,-0.77
5,-8.839419,-8.482415,-7.511965,-9.684818,-7.550761,-7.723528,-5.972520,-7.761362,-7.318130,-7.491262,196506,1.348157,-5.51,-4.35,0.59,0.35,-3.13,-5.51
6,1.117375,2.229268,1.815703,4.744770,2.564105,2.315936,2.306756,2.158138,2.583090,0.878024,196507,-0.239351,1.43,0.87,2.14,0.31,4.12,1.43
7,3.182139,4.563220,3.102199,6.080565,4.558191,4.466981,3.174998,4.178075,4.303364,5.074235,196508,1.892095,2.73,2.83,-1.02,0.33,2.58,2.73
8,3.716342,5.886347,4.934096,4.424657,3.635884,3.077862,2.833461,2.855150,3.393569,3.511430,196509,-0.204912,2.86,0.64,-0.12,0.31,3.30,2.86
9,2.390831,5.019396,3.442919,5.949287,4.007861,3.995972,3.073679,4.481654,3.868764,3.175576,196510,0.784745,2.60,2.53,1.57,0.31,3.46,2.60


In [6]:
i = 0
for i in range(11):
    pi = cls[i]
    mean = round(df_pf_rets2[pi].mean(), 2)
    std = round(df_pf_rets2[pi].std(), 2)

    Y = df_pf_rets2[pi]
    X = df_pf_rets2['MK']
    X = sm.add_constant(X)
    mod_ff = sm.OLS(Y, X)
    ols_ff = mod_ff.fit()

    tvs = round(ols_ff.tvalues, 2)
    t1 = round(tvs[0], 2)
    t2 = round(tvs[1], 2)
    coefs = ols_ff.params
    coef1 = round(coefs[0], 2)
    coef2 = round(coefs[1], 2)

    AdjR2 = round(ols_ff.rsquared_adj, 2)

    X1 = df_pf_rets2[['MK', 'SMB', 'HML', 'Mom']]
    X1 = sm.add_constant(X1)
    mod_ff1 = sm.OLS(Y, X1)
    ols_ff1 = mod_ff1.fit()
    tvs1 = ols_ff1.tvalues
    t3 = round(tvs1[0], 2)
    t4 = round(tvs1[1], 2)
    t5 = round(tvs1[2], 2)
    t6 = round(tvs1[3], 2)
    t7 = round(tvs1[4], 2)

    coefs1 = ols_ff1.params
    coef11 = round(coefs1[0], 2)
    coef12 = round(coefs1[1], 2)
    coef13 = round(coefs1[2], 2)
    coef14 = round(coefs1[3], 2)
    coef15 = round(coefs1[4], 2)

    AdjR21 = round(ols_ff1.rsquared_adj, 2)
    line1 = '{}&{}\%&{}\%&{}\%&{}&{}&{}\%&{}&{}&{}&{}&{}\\\\'.format(i+1, mean, std, coef1, coef2, AdjR2, coef11, coef12, coef13, coef14, coef15, AdjR21)
    line2 = '&&&({})&({})&&({})&({})&({})&({})&({})&\\\\'.format(t1, t2, t3, t4, t5, t6, t7)
    print(line1)
    print(line2)

1&0.77\%&5.4\%&0.37\%&1.03&0.75&-0.05\%&0.87&0.62&-0.04&0.34&0.88\\
&&&(2.58)&(32.6)&&(-0.44)&(34.76)&(17.02)&(-0.93)&(11.87)&\\
2&0.64\%&5.22\%&0.23\%&1.06&0.86&0.03\%&0.92&0.44&-0.16&0.21&0.93\\
&&&(2.17)&(46.41)&&(0.35)&(50.59)&(16.66)&(-5.29)&(10.05)&\\
3&0.49\%&5.12\%&0.08\%&1.05&0.88&-0.09\%&0.93&0.38&-0.17&0.2&0.95\\
&&&(0.87)&(51.48)&&(-1.33)&(57.07)&(15.99)&(-6.17)&(10.64)&\\
4&0.42\%&4.98\%&0.02\%&1.03&0.9&-0.14\%&0.92&0.36&-0.15&0.18&0.96\\
&&&(0.2)&(56.06)&&(-2.39)&(64.56)&(17.29)&(-6.23)&(10.93)&\\
5&0.47\%&4.83\%&0.07\%&1.03&0.94&-0.03\%&0.94&0.27&-0.12&0.12&0.97\\
&&&(1.14)&(73.63)&&(-0.62)&(85.02)&(16.48)&(-6.81)&(9.66)&\\
6&0.33\%&4.72\%&-0.06\%&1.0&0.93&-0.12\%&0.91&0.25&-0.13&0.08&0.96\\
&&&(-0.92)&(68.15)&&(-2.07)&(69.08)&(12.91)&(-6.2)&(5.19)&\\
7&0.33\%&4.91\%&-0.08\%&1.04&0.94&-0.11\%&0.96&0.27&-0.09&0.02&0.96\\
&&&(-1.16)&(72.1)&&(-2.02)&(72.86)&(13.95)&(-3.95)&(1.58)&\\
8&0.27\%&4.86\%&-0.13\%&1.03&0.93&-0.11\%&0.94&0.24&-0.14&0.0&0.95\\
&&&(-1.93)&(68.16)&&(-1

In [7]:
i = 0
for i in range(11):
    pi = cls[i]
    mean = round(df_pf_rets3[pi].mean(), 2)
    std = round(df_pf_rets3[pi].std(), 2)

    Y = df_pf_rets3[pi]
    X = df_pf_rets3['MK']
    X = sm.add_constant(X)
    mod_ff = sm.OLS(Y, X)
    ols_ff = mod_ff.fit()

    tvs = round(ols_ff.tvalues, 2)
    t1 = round(tvs[0], 2)
    t2 = round(tvs[1], 2)
    coefs = ols_ff.params
    coef1 = round(coefs[0], 2)
    coef2 = round(coefs[1], 2)

    AdjR2 = round(ols_ff.rsquared_adj, 2)

    X1 = df_pf_rets3[['MK', 'SMB', 'HML', 'Mom']]
    X1 = sm.add_constant(X1)
    mod_ff1 = sm.OLS(Y, X1)
    ols_ff1 = mod_ff1.fit()
    tvs1 = ols_ff1.tvalues
    t3 = round(tvs1[0], 2)
    t4 = round(tvs1[1], 2)
    t5 = round(tvs1[2], 2)
    t6 = round(tvs1[3], 2)
    t7 = round(tvs1[4], 2)

    coefs1 = ols_ff1.params
    coef11 = round(coefs1[0], 2)
    coef12 = round(coefs1[1], 2)
    coef13 = round(coefs1[2], 2)
    coef14 = round(coefs1[3], 2)
    coef15 = round(coefs1[4], 2)

    AdjR21 = round(ols_ff1.rsquared_adj, 2)
    line1 = '{}&{}\%&{}\%&{}\%&{}&{}&{}\%&{}&{}&{}&{}&{}\\\\'.format(i+1, mean, std, coef1, coef2, AdjR2, coef11, coef12, coef13, coef14, coef15, AdjR21)
    line2 = '&&&({})&({})&&({})&({})&({})&({})&({})&\\\\'.format(t1, t2, t3, t4, t5, t6, t7)
    print(line1)
    print(line2)


1&0.62\%&5.31\%&-0.02\%&1.03&0.69&-0.17\%&1.02&0.49&-0.12&0.29&0.89\\
&&&(-0.11)&(25.98)&&(-1.64)&(40.06)&(14.98)&(-3.33)&(13.04)&\\
2&0.61\%&4.68\%&-0.03\%&1.02&0.87&-0.12\%&1.01&0.33&0.04&0.14&0.95\\
&&&(-0.26)&(44.93)&&(-1.95)&(64.64)&(16.44)&(2.0)&(10.71)&\\
3&0.58\%&4.44\%&-0.05\%&1.0&0.95&-0.11\%&1.0&0.2&0.07&0.08&0.97\\
&&&(-0.77)&(72.51)&&(-2.47)&(92.78)&(14.07)&(4.65)&(8.08)&\\
4&0.52\%&4.3\%&-0.1\%&0.98&0.97&-0.14\%&0.98&0.13&0.07&0.04&0.98\\
&&&(-2.18)&(94.19)&&(-3.84)&(108.64)&(11.53)&(5.7)&(5.41)&\\
5&0.5\%&4.13\%&-0.09\%&0.95&0.98&-0.11\%&0.94&0.07&0.07&-0.0&0.98\\
&&&(-2.51)&(111.58)&&(-3.06)&(110.96)&(6.14)&(5.55)&(-0.32)&\\
6&0.45\%&4.14\%&-0.14\%&0.95&0.97&-0.13\%&0.93&0.05&0.04&-0.04&0.98\\
&&&(-3.69)&(106.19)&&(-3.47)&(103.34)&(4.34)&(2.8)&(-4.9)&\\
7&0.43\%&4.17\%&-0.16\%&0.95&0.96&-0.13\%&0.92&0.07&0.06&-0.07&0.97\\
&&&(-3.15)&(80.39)&&(-2.92)&(81.24)&(4.54)&(3.97)&(-6.63)&\\
8&0.42\%&4.16\%&-0.16\%&0.93&0.93&-0.13\%&0.91&0.06&0.08&-0.07&0.94\\
&&&(-2.54)&(62.95)&

In [8]:
i = 0
for i in range(11):
    pi = cls[i]
    mean = round(df_pf_rets1[pi].mean(), 2)
    std = round(df_pf_rets1[pi].std(), 2)

    Y = df_pf_rets1[pi]
    X = df_pf_rets1['MK']
    X = sm.add_constant(X)
    mod_ff = sm.OLS(Y, X)
    ols_ff = mod_ff.fit()

    tvs = round(ols_ff.tvalues, 2)
    t1 = round(tvs[0], 2)
    t2 = round(tvs[1], 2)
    coefs = ols_ff.params
    coef1 = round(coefs[0], 2)
    coef2 = round(coefs[1], 2)

    AdjR2 = round(ols_ff.rsquared_adj, 2)

    X1 = df_pf_rets1[['MK', 'SMB', 'HML', 'Mom']]
    X1 = sm.add_constant(X1)
    mod_ff1 = sm.OLS(Y, X1)
    ols_ff1 = mod_ff1.fit()
    tvs1 = ols_ff1.tvalues
    t3 = round(tvs1[0], 2)
    t4 = round(tvs1[1], 2)
    t5 = round(tvs1[2], 2)
    t6 = round(tvs1[3], 2)
    t7 = round(tvs1[4], 2)

    coefs1 = ols_ff1.params
    coef11 = round(coefs1[0], 2)
    coef12 = round(coefs1[1], 2)
    coef13 = round(coefs1[2], 2)
    coef14 = round(coefs1[3], 2)
    coef15 = round(coefs1[4], 2)

    AdjR21 = round(ols_ff1.rsquared_adj, 2)
    line1 = '{}&{}\%&{}\%&{}\%&{}&{}&{}\%&{}&{}&{}&{}&{}\\\\'.format(i+1, mean, std, coef1, coef2, AdjR2, coef11, coef12, coef13, coef14, coef15, AdjR21)
    line2 = '&&&({})&({})&&({})&({})&({})&({})&({})&\\\\'.format(t1, t2, t3, t4, t5, t6, t7)
    print(line1)
    print(line2)


1&0.7\%&5.36\%&0.19\%&1.03&0.73&-0.06\%&0.94&0.55&-0.07&0.29&0.88\\
&&&(1.72)&(41.32)&&(-0.81)&(53.04)&(22.41)&(-2.39)&(16.67)&\\
2&0.63\%&4.98\%&0.11\%&1.04&0.86&-0.04\%&0.98&0.36&-0.04&0.16&0.93\\
&&&(1.48)&(64.19)&&(-0.71)&(79.36)&(21.2)&(-1.86)&(13.01)&\\
3&0.53\%&4.81\%&0.02\%&1.03&0.91&-0.08\%&0.99&0.25&-0.02&0.11&0.94\\
&&&(0.33)&(79.9)&&(-1.81)&(90.08)&(16.67)&(-1.18)&(10.23)&\\
4&0.46\%&4.67\%&-0.04\%&1.01&0.93&-0.12\%&0.98&0.21&-0.01&0.08&0.95\\
&&&(-0.8)&(89.97)&&(-2.9)&(97.54)&(15.27)&(-0.5)&(8.42)&\\
5&0.49\%&4.52\%&-0.01\%&0.99&0.95&-0.05\%&0.97&0.14&-0.0&0.04&0.96\\
&&&(-0.23)&(113.43)&&(-1.48)&(113.83)&(11.7)&(-0.28)&(4.86)&\\
6&0.38\%&4.46\%&-0.1\%&0.98&0.95&-0.11\%&0.95&0.12&-0.03&0.0&0.95\\
&&&(-2.54)&(107.86)&&(-2.74)&(102.97)&(9.58)&(-1.81)&(0.03)&\\
7&0.38\%&4.58\%&-0.12\%&1.0&0.94&-0.12\%&0.97&0.15&0.01&-0.03&0.95\\
&&&(-2.81)&(102.84)&&(-2.9)&(99.66)&(10.84)&(0.64)&(-3.3)&\\
8&0.34\%&4.54\%&-0.15\%&0.99&0.93&-0.13\%&0.95&0.12&-0.01&-0.05&0.94\\
&&&(-3.2)&(91.49)